In [ ]:
# | default_exp vis

# AIP Visualization

> Tools to visualize AIP


In [ ]:
# | hide
from nbdev.showdoc import *


In [ ]:
# | export
import boto3
import os
import py7zr
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
from glob import glob
from mets_tools.core import METSFile

In [ ]:
# | export
class VisClient:
    """
    Client for visualizing AIPs.
    """

    @staticmethod
    def main(uuid, env_path=None):
        """
        Main function to visualize an AIP.

        Args:
            uuid: The UUID of the AIP to visualize.
            env_path: The path to the environment file.
        Returns:
        """

        if env_path:
            load_dotenv(override=True, dotenv_path=env_path)
        else:
            load_dotenv(override=True)

        endpoint_url = os.getenv("OS_ENDPOINT")
        aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
        aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
        bucket_name = os.getenv("OS_BUCKET_NAME")
        aip_prefix = os.getenv("OS_AIP_PREFIX")
        
        vis = VisClient(endpoint_url, aws_access_key_id, aws_secret_access_key, bucket_name, aip_prefix)
        ins = vis.visualize(uuid)
        return ins

    def __init__(self, endpoint_url: str, aws_access_key_id: str, aws_secret_access_key: str, bucket_name: str, aip_prefix: str):
        """
        Initialize the VisClient.

        Args:
            endpoint_url: The URL of the endpoint.
            aws_access_key_id: The access key ID for AWS.
            aws_secret_access_key: The secret access key for AWS.
            bucket_name: The name of the bucket.
            aip_prefix: The prefix of the AIP.
        """
        self.endpoint_url = endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.bucket_name = bucket_name
        self.aip_prefix = aip_prefix

    def visualize(self, uuid):
        """
        Visualize an AIP.

        Args:
            uuid: The UUID of the AIP to visualize.

        Returns:
        """
        local_file = self.download_data(uuid, self.bucket_name)
        ins = METSFile(local_file)
        ins.parse_mets()

        return ins
    
    def download_zip(self, uuid, bucket_name, zip_dir = "./zip_dir"):
        """
        Download the zip file for an AIP.

        Args:
            uuid: The UUID of the AIP to download.
            bucket_name: The name of the bucket.
            zip_dir: The directory to save the zip file.

        Returns:
            The path to the zip file.
        """

        uuid_cleaned = uuid.replace('-', '')
        chunks_cleaned = [uuid_cleaned[i:i+4] for i in range(0, len(uuid_cleaned), 4)]
        # chunks_cleaned
        
        key_prefix = f"{self.aip_prefix}/{'/'.join(chunks_cleaned)}/"
        
        s3_client = boto3.client(
            's3',
            endpoint_url=self.endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key
        )
        
        # 指定されたプレフィックス内のファイルをリストアップ
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=key_prefix)

        s3_key = ""
        
        # ファイルリストを表示
        if 'Contents' in response:
            for obj in response['Contents']:
                # print(obj['Key'])
                s3_key = obj['Key']
        else:
            print('指定のプレフィックス内にファイルが見つかりませんでした。')

        if s3_key == "":
            return
        
        # ダウンロード先のローカルファイルパス
        local_file = zip_dir + "/" + s3_key.split("/")[-1]

        os.makedirs(os.path.dirname(local_file), exist_ok=True)
        
        # S3からファイルをダウンロード
        s3_client.download_file(bucket_name, s3_key, local_file)
        print(f"ファイル {local_file} をダウンロードしました。")

        # pass

        return local_file, s3_key

    def download_data(self, uuid, bucket_name, zip_dir = "./zip_dir", extract_dir = './extracted_files'):
        """
        Download the data for an AIP.

        Args:
            uuid: The UUID of the AIP to download.
            bucket_name: The name of the bucket.
            zip_dir: The directory to save the zip file.
            extract_dir: The directory to save the extracted files.

        Returns:
            The path to the extracted METS file.
        """
        
        local_file, s3_key = self.download_zip(uuid, bucket_name, zip_dir)
        
        # 展開先のディレクトリを指定
        
        os.makedirs(extract_dir, exist_ok=True)

        # .7zファイルを展開
        with py7zr.SevenZipFile(local_file, mode='r') as archive:
            archive.extractall(path=extract_dir)
            print(f"ファイルを {extract_dir} に展開しました。")

        return extract_dir + "/" + s3_key.split("/")[-1].replace(".7z", "") + "/data/METS." + uuid + ".xml"

In [ ]:
show_doc(VisClient)

---

[source](https://github.com/nakamura196/archivematica_tools/blob/main/archivematica_tools/vis.py#L17){target="_blank" style="float:right; font-size:smaller"}

### VisClient

>      VisClient (endpoint_url:str, aws_access_key_id:str,
>                 aws_secret_access_key:str, bucket_name:str, aip_prefix:str)

Client for visualizing AIPs.

In [ ]:
show_doc(VisClient.main)

---

[source](https://github.com/nakamura196/archivematica_tools/blob/main/archivematica_tools/vis.py#L23){target="_blank" style="float:right; font-size:smaller"}

### VisClient.main

>      VisClient.main (uuid, env_path=None)

Main function to visualize an AIP.

Args:
    uuid: The UUID of the AIP to visualize.
    env_path: The path to the environment file.
Returns:

In [ ]:
show_doc(VisClient.download_data)

---

[source](https://github.com/nakamura196/archivematica_tools/blob/main/archivematica_tools/vis.py#L80){target="_blank" style="float:right; font-size:smaller"}

### VisClient.download_data

>      VisClient.download_data (uuid, bucket_name, zip_dir='./zip_dir',
>                               extract_dir='./extracted_files')

Download the data for an AIP.

Args:
    uuid: The UUID of the AIP to download.
    bucket_name: The name of the bucket.
    zip_dir: The directory to save the zip file.
    extract_dir: The directory to save the extracted files.

Returns:
    The path to the extracted METS file.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()